In [4]:
%matplotlib inline
import cv2
import coremltools
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
import keras
from keras.models import model_from_json

import PIL
from PIL import Image

In [8]:
print("tensorflow version : ",tf.__version__)
print("keras version : ",keras.__version__)
print("coreml version : ",coremltools.SPECIFICATION_VERSION)

tensorflow version :  1.5.0
keras version :  2.1.3
coreml version :  1


In [ ]:
with open("model.json",'r') as f:
    model = model_from_json(f.read())
model.load_weights("model.h5")   

In [ ]:
# Sample Image
img = cv2.imread("sample.png")
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
img = img / 255.

In [ ]:
out = model.predict(np.expand_dims(img,axis=0))

plt.imshow(img)
plt.show()
plt.imshow(out[0,:,:,:3])
plt.show()

print("NAN Counts in Keras.predict :",np.isnan(out.flatten()).sum())

In [ ]:
for _ in range(20):
    coreml_model = coremltools.converters.keras.convert(
        model,
        input_names='image',
        image_input_names='image',
        output_names='output',
        image_scale=1/255.0,
    )

    coreml_inputs = {"image":Image.fromarray((img*255).astype(np.uint8))}
    output = coreml_model.predict(coreml_inputs,useCPUOnly=True)['output']
    
    print("- "*50)
    count=0
    for i in range(output.shape[0]):
        nb_nan = np.isnan(output[i].flatten()).sum()
        if nb_nan > 0:
            print("{:2d}th layer have {} nans".format(i,nb_nan))
            count+=1
    print("\ntotal NaN layer : {}".format(count))
    print("- "*50)